# Simple Time Series Activity Index Example

This example shows how the Capella API can be used to fetch a time series stack of data, read data for a given AOI using Rasterio masks directly from cloud optimized geotiffs stored in Capella's S3 bucket, examine the histogram, use Otsu thresholding to count the number of bright pixels for each image in the time series, and plot the normalized count over time to give an indication of activity. 

To run this notebook, you will need a Capella API account, with credentials saved in a credentials.json file.

In [ ]:
import capella
from capella import lee_filter

import json

import folium
from IPython.display import HTML
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import rasterio
from rasterio.plot import show_hist
from rasterio.plot import show
from rasterio.windows import Window
from rasterio import warp
from rasterio import mask
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage import exposure
from skimage import filters

### Set up project variables

In [ ]:
with open('filter-timeseries.json') as f:
    filter = json.load(f)
    aoi = filter['geometry']

# Windows sizes for filtering
FILTSIZE = 3 # window size for speckle filter

### Inspect the AOI on a map

The AOI used for this demonstration is a container staging area at DFDS Seaways B.V. terminal in the port of Rotterdam

In [ ]:
# Display the results on a folium map
m = folium.Map(location=[51.90911331244101, 4.364436864852905],zoom_start=15)
folium.GeoJson(
    aoi,
    name='AOI'
).add_to(m)

m

### Use the API to search for Capella SAR data

In [ ]:
result = ! rio capella --credentials credentials.json --area filter-gif.json --collection rotterdam-aerial-mosaic --limit 50 query
fc = json.loads(result[0])
features = fc['features']

In [ ]:
# Sort the results in time
features =  sorted(features, key = lambda f: f['properties']['datetime'])

### Open the first image in the time series and inspect the histogram

To find the right threshold between dark and bright pixels, we can inspect the histogram for the masked area.

Note that the AOI mask is being used to read only pixels within the AOI

In [ ]:
# Open the file with Rasterio
f= features[0]
with rasterio.open(f"tiledb://capellaspace/{f['id']}") as src:
    meta = src.meta
    roi_polygon_src_coords = warp.transform_geom({'init': 'epsg:4326'}, src.crs, aoi)
    out_image, out_transform = mask.mask(src, [roi_polygon_src_coords], crop=True)
    out_image[out_image == meta['nodata']] = 0

thresh = filters.threshold_otsu(out_image) #Use Otsu to set a threshold
plt.hist(out_image.ravel(),bins=256, range=(out_image.min(), out_image.max()), fc='k', ec='k')
plt.axvline(thresh, color='k', ls='--')
plt.title("Histogram")
plt.ylabel('Pixel Count')
plt.xlabel('DN')
plt.show()

### Set a threshold and count pixels above the threshold

We can now set a threshold and for each image in the time series count the number of pixels above the threshold.

In [ ]:
activity = {}

for f in features:
    with rasterio.open(f"tiledb://capellaspace/{f['id']}") as src:
        meta = src.meta
        roi_polygon_src_coords = warp.transform_geom({'init': 'epsg:4326'}, src.crs,aoi)
        out_image, out_transform = mask.mask(src,[roi_polygon_src_coords], crop=True)
        out_image[out_image == meta['nodata']] = 0
        thresh1 = np.where(out_image > thresh, 70000, 0)
        activity.update( {f['properties']['datetime'] : np.count_nonzero(thresh1)} )

lists = sorted(activity.items())
x, y = zip(*lists) # unpack a list of pairs into two tuples
y = list(y)
y = [(i - min(y)) / (max(y) - min(y)) for i in y] # normalize the pixel values to an index
y = tuple(y)

In [ ]:
fix, ax = plt.subplots()

ax.set_title('Activity Over Time')
ax.set_ylabel('Activity Index')
ax.set_xticklabels(x, rotation=70)
ax.plot(x, y)
plt.show()

### Compare the activity graph to the time series imagery

We can now inspect the imagery to see if the activity shown in the graph correlates with the imagery. The bright yellow blobs in the imagery are shipping containers in this case. 

In [ ]:
# Show the imagery alongside the graph for validation
timeseries = []

rcParams['figure.figsize'] = 12, 8

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.set_title('Activity over Time')
ax1.set_ylabel('Activity Index')
ax1.set_xticklabels(x, rotation=70)
ax2.set_title('SAR image')
activity, = ax1.plot(x, y)


for idx, f in enumerate(features):
    with rasterio.open(f"tiledb://capellaspace/{f['id']}") as src:
        timestamp = f['properties']['datetime']
        meta = src.meta
        roi_polygon_src_coords = warp.transform_geom({'init': 'epsg:4326'}, src.crs,aoi)
        out_img, out_transform = mask.mask(src, [roi_polygon_src_coords], crop=True, indexes=1)
        out_img[out_img == meta['nodata']] = 0
        out_img = lee_filter(out_img, FILTSIZE)
        out_img = exposure.adjust_log(out_img, gain=10)
    
        out_img_min = out_img.min()
        out_img_max = out_img.max()
        out_img_scaled = (out_img - out_img_min) / (out_img_max - out_img_min) * 255

        marker, = ax1.plot(x[idx], y[idx], 'ro')
        frame =  ax2.imshow(out_img_scaled, cmap='gray')
        t = ax2.annotate(f["properties"]["datetime"], (10, 10), color='red', fontsize=15) # add text
        timeseries.append([activity, marker, frame, t])

anim = animation.ArtistAnimation(fig, timeseries, interval=350, blit=True, repeat_delay=350)
plt.close()
HTML(anim.to_html5_video())

In [ ]:
anim.save('animation.gif', writer='imagemagick', fps=5)